In [1]:
from openai import OpenAI
import re

In [2]:
client =  OpenAI(
    api_key="YOUR_API_KEY"
)

In [3]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content

In [4]:
prompt = """
You will follow a loop of Thought, Action, PAUSE and Response. The goal is to get an answer at the end.
    Thought: Think about how to answer the question.
    Action: Use one of the available actions to get information - then return PAUSE.
    Response: After the action is complete, respond with the result.

Available actions:
    lookup_capital: Finds the capital city of a country. Example: lookup_capital: France, Result: Returns the capital city (e.g., "Paris").
    find_population: Looks up the population of a country or city. Example: find_population: Tokyo, Result: Returns the population (e.g., "14 million").

Example conversation:

Question: What is the population of capital city of France?

    Thought: I should find the capital city of France.
    Action: lookup_capital: Paris
    PAUSE

Then you'll find the population of the capital city:

Response: The population of Paris is approximately 2.1 million people.

Finally, respond:

Answer: The population of Paris is approximately 2.1 million people.
""".strip()

In [5]:
def lookup_capital(country):
    capitals = {
        "France": "Paris",
        "Germany": "Berlin",
        "Japan": "Tokyo",
        "USA": "Washington, D.C.",
        "Canada": "Ottawa",
    }
    return capitals.get(country, "Capital not found for {}".format(country))

def find_population(city):
    populations = {
        "Tokyo": "14 million",
        "New York": "8.3 million",
        "London": "9 million",
        "Paris": "2.1 million",
        "Sydney": "5.3 million",
    }
    return populations.get(city, "Population data not found for {}".format(city))

known_actions = {
    "find_population": find_population,
    "lookup_capital": lookup_capital
}

In [6]:
action_re = re.compile('^Action: (\w+): (.*)$')  

In [7]:
def query(question, max_turns=5):
    i = 0
    aiagent = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = aiagent(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            response = known_actions[action](action_input)
            print("Response:", response)
            next_prompt = "Response: {}".format(response)
        else:
            return

In [8]:
def main():
    print("Welcome to the Custom AI agent! Type 'exit' to end the conversation.")
    aiagent = Agent(prompt)
    agent_input = input("You: ")
    while True:

        # Check for the exit condition
        if agent_input.lower() == "exit":
            print("Goodbye!")
            break

        response = aiagent(agent_input)
        agent_input = response
        print("AI: " + response)
        if "Answer: " in response:
            print("If you have any more questions or need further assistance, feel free to ask.")
            agent_input = input("You: ")

In [9]:
if __name__ == "__main__":
    main()

Welcome to the Custom AI agent! Type 'exit' to end the conversation.


You:  What's the population of the capital city of Japan?


AI:     Thought: I need to find the capital city of Japan first.
    Action: lookup_capital: Japan
    PAUSE
AI: Response: The capital city of Japan is Tokyo.

    Thought: Now I need to find the population of Tokyo.
    Action: find_population: Tokyo
    PAUSE
AI: Response: The population of Tokyo is approximately 14 million people.

Answer: The population of Tokyo is approximately 14 million people.
If you have any more questions or need further assistance, feel free to ask.


You:  What's the population of capital city of France?


AI:     Thought: I need to find the capital city of France first.
    Action: lookup_capital: France
    PAUSE
AI: Response: The capital city of France is Paris.

    Thought: Now I need to find the population of Paris.
    Action: find_population: Paris
    PAUSE
AI: Response: The population of Paris is approximately 2.1 million people.

Answer: The population of Paris is approximately 2.1 million people.
If you have any more questions or need further assistance, feel free to ask.


You:  exit


Goodbye!
